# Notebook to edit metadata for Qiita: Cancer dataset from Liao
Guide: https://qiita.ucsd.edu/static/doc/html/checklist-for-ebi-ena-submission.html#checklist

# Questions:
- Do we want to add in from here: https://www.nature.com/articles/s41597-021-00860-8 or should we just link the paper?
    - tblbc.csv: Daily measurements of white blood cells, platelets and red blood cells for 1,278 patients
    - tbltemperature.csv: temperatures for 1,249 patients
    - tblInfectionsCidPapers.csv: The day of positive blood cultures for 426 patients and microbes (genera Enterococcus, Escherichia, Klebsiella, Enterobacter, Pseudomonas, Stenotrophomonas, and Citrobacter) analysed in previous publications from our team
    - tbldrug.csv: Timing and route of drug administration for 1,278 patients
    - tblhctmeta.csv: The day and source of HCT for 1,278 patients
    
- If we follow the sample info exactly, (https://qiita.ucsd.edu/static/doc/html/checklist-for-ebi-ena-submission.html#checklist) we are missing the following: 
    - elevation, lat, long
    - physical_specimen_location
    - collection_timestamp
        - We have a de-identifyed time stamp, so similar
        
- If we follow the prep info exactly, (https://qiita.ucsd.edu/static/doc/html/checklist-for-ebi-ena-submission.html#checklist) we are missing the following:
    - barcode
        - Only info I could find in the paper: A unique 12-base Golay barcode (Ns) precedes the primers for sample identification, and one to eight additional nucleotides were placed in front of the barcode to offset the sequencing of the primers. 
    - Two sets of primers: primer designed to amplify the V4-V5: 563 F (5′-nnnnnnnn-NNNNNNNNNNNN-AYTGGGYDTAAAGNG-3′) and 926 R (5′-nnnnnnnn-NNNNNNNNNNNN-CCGTCAATTYHTTTRAGT-3′). 
        - Do these need to be run as two different studies? I've never had a study with two primers before
    - Experimental_design_description
    - Center name 
    - center_project_name
    - library_construction_protocol
- Under Patient_ID some are labled 'pt_with_samples_ID'
    - I'm not sure what this means so I assumed this was still a sample, but wasn't sure if these were controls?
- Was there any particular reason some of the Patient_IDs were labled with 'FMT' to start, should these be pooled?

In [1]:
import pandas as pd
import random

### Import file paths

In [2]:
fn_path = '22_07_13_QiitaCancer_Liao_filenames.txt'
meta_path = 'tblASVsamples_06222022.csv'

### Convert into list/df

In [3]:
#Filename list
fn = []
with open(fn_path) as file:
    while (line := file.readline().rstrip()):
        fn.append(line)
print(fn[:3])

#Metadata df
meta = pd.read_csv(meta_path)
meta[:3]

['1000A..pool483_R1.fastq.gz', '1000A..pool483_R2.fastq.gz', '1000B..pool483_R1.fastq.gz']


,SampleID,PatientID,Timepoint,Consistency,Accession,BioProject,DayRelativeToNearestHCT,AccessionShotgun,Pool
0,1000A,1000,0,formed,SRR11414397,PRJNA545312,-9.0,NaN,483
1,1000B,1000,5,liquid,SRR11414992,PRJNA545312,-4.0,NaN,483
2,1000C,1000,15,liquid,SRR11414991,PRJNA545312,6.0,NaN,483


## Sample Information

In [4]:
s = meta.copy()

#sample_name
s.rename(columns = {'SampleID':'sample_name'}, inplace = True)

#host_subject_id
s.rename(columns = {'PatientID':'host_subject_id'}, inplace = True)

#taxon_id and scientific_name
s["taxon_id"] = 256318
s["scientific_name"] = 'metagenome'

#env
s["env_biome"] = 'urban biome'
s["env_feature"] = 'human-associated habitat'
s["env_material"] = 'feces'
s["env_package"] = 'human-gut'

#empo
s["empo_1"] = 'Host-associated'
s["empo_2"] = 'Animal-associated'
s["empo_3"] = 'Animal distal gut'

s[:3]

,sample_name,host_subject_id,Timepoint,Consistency,Accession,BioProject,DayRelativeToNearestHCT,AccessionShotgun,Pool,taxon_id,scientific_name,env_biome,env_feature,env_material,env_package,empo_1,empo_2,empo_3
0,1000A,1000,0,formed,SRR11414397,PRJNA545312,-9.0,NaN,483,256318,metagenome,urban biome,human-associated habitat,feces,human-gut,Host-associated,Animal-associated,Animal distal gut
1,1000B,1000,5,liquid,SRR11414992,PRJNA545312,-4.0,NaN,483,256318,metagenome,urban biome,human-associated habitat,feces,human-gut,Host-associated,Animal-associated,Animal distal gut
2,1000C,1000,15,liquid,SRR11414991,PRJNA545312,6.0,NaN,483,256318,metagenome,urban biome,human-associated habitat,feces,human-gut,Host-associated,Animal-associated,Animal distal gut


## Prep Information

In [5]:
p = meta.copy()

#sample_name
s.rename(columns = {'SampleID':'sample_name'}, inplace = True)

#run_prefix
p["run_prefix"] = p["SampleID"] + str('..pool') + p["Pool"]

#instrument_model
p["instrument_model"] = 'Illumina MiSeq'

#platform
p["platform"] = 'Illumina'

#target_gene
p['target_gene'] = '16S rRNA'

p.drop(['Timepoint', 'Consistency', 'DayRelativeToNearestHCT'], inplace=True, axis=1)
print(len(p))
p[:3]

12535


,SampleID,PatientID,Accession,BioProject,AccessionShotgun,Pool,run_prefix,instrument_model,platform,target_gene
0,1000A,1000,SRR11414397,PRJNA545312,NaN,483,1000A..pool483,Illumina MiSeq,Illumina,16S rRNA
1,1000B,1000,SRR11414992,PRJNA545312,NaN,483,1000B..pool483,Illumina MiSeq,Illumina,16S rRNA
2,1000C,1000,SRR11414991,PRJNA545312,NaN,483,1000C..pool483,Illumina MiSeq,Illumina,16S rRNA


### Split Prep Information

In [6]:
#Split FMT. cases out
fmt = p[p.apply(lambda x: 'FMT' in x.PatientID, axis=1)]
fmt = fmt.reset_index(drop=True)

print(len(fmt))
display(fmt[:3])
print(fmt.loc[:]['Pool'].value_counts())

3408


,SampleID,PatientID,Accession,BioProject,AccessionShotgun,Pool,run_prefix,instrument_model,platform,target_gene
0,1004A,FMT.0002,SRR11414975,PRJNA545312,NaN,483,1004A..pool483,Illumina MiSeq,Illumina,16S rRNA
1,1004B,FMT.0002,SRR11414972,PRJNA545312,NaN,483,1004B..pool483,Illumina MiSeq,Illumina,16S rRNA
2,1083A,FMT.0023,SRR11414929,PRJNA545312,NaN,550,1083A..pool550,Illumina MiSeq,Illumina,16S rRNA


735        68
621        59
698        58
761        58
730        57
           ..
516.BMT     1
500         1
762         1
553         1
668.fmt     1
Name: Pool, Length: 151, dtype: int64


In [7]:
#Randomly split remaning cases keeping pool numbers aligned

#Change this variable depending on the size per group
aprx_size_lists = 500

pool_df = fmt.loc[:]['Pool'].value_counts() 
all_pools = list(pool_df.index)
pool_groups = []

while len(all_pools) > int(aprx_size_lists/50):
    current_group = []
    sum_current_group = 0
    while sum_current_group < aprx_size_lists and len(all_pools)>=1:
        selected_pool = random.sample(all_pools, 1)
        count_in_pool = pool_df[selected_pool][0]
        sum_current_group += count_in_pool
        current_group.append(selected_pool[0])
        all_pools.remove(selected_pool[0])
    pool_groups.append(current_group)
    #print(sum_current_group)
print('Total number of groups:', len(pool_groups), 'of ~', aprx_size_lists, 'samples')
fmt_pool_groups = pool_groups

Total number of groups: 6 of ~ 500 samples


In [8]:
#Split pt cases out
pt = p[p.apply(lambda x: 'pt' in x.PatientID, axis=1)]
pt = pt.reset_index(drop=True)
print(len(pt))
display(pt[:3])

186


,SampleID,PatientID,Accession,BioProject,AccessionShotgun,Pool,run_prefix,instrument_model,platform,target_gene
0,1001,pt_with_samples_1001_1002_1003_1004_1005_1006_...,SRR11414988,PRJNA545312,NaN,535,1001..pool535,Illumina MiSeq,Illumina,16S rRNA
1,1002,pt_with_samples_1001_1002_1003_1004_1005_1006_...,SRR11414981,PRJNA545312,NaN,535,1002..pool535,Illumina MiSeq,Illumina,16S rRNA
2,1005,pt_with_samples_1001_1002_1003_1004_1005_1006_...,SRR11414971,PRJNA545312,NaN,203,1005..pool203,Illumina MiSeq,Illumina,16S rRNA


In [9]:
#Remaining cases (w/o FMT and pt)
p0 = p[p.apply(lambda x: 'pt' not in x.PatientID, axis=1)]
p0 = p0[p0.apply(lambda x: 'FMT' not in x.PatientID, axis=1)]
p0 = p0.reset_index(drop=True)
print(len(p0))
print(8941 + 186 + 3408)
display(p0[:3])

8941
12535


,SampleID,PatientID,Accession,BioProject,AccessionShotgun,Pool,run_prefix,instrument_model,platform,target_gene
0,1000A,1000,SRR11414397,PRJNA545312,NaN,483,1000A..pool483,Illumina MiSeq,Illumina,16S rRNA
1,1000B,1000,SRR11414992,PRJNA545312,NaN,483,1000B..pool483,Illumina MiSeq,Illumina,16S rRNA
2,1000C,1000,SRR11414991,PRJNA545312,NaN,483,1000C..pool483,Illumina MiSeq,Illumina,16S rRNA


In [10]:
#Randomly split remaning cases keeping pool numbers aligned

#Change this variable depending on the size per group
aprx_size_lists = 500

pool_df = p0.loc[:]['Pool'].value_counts() 
all_pools = list(pool_df.index)
pool_groups = []

while len(all_pools) > int(aprx_size_lists/50):
    current_group = []
    sum_current_group = 0
    while sum_current_group < aprx_size_lists and len(all_pools)>=1:
        selected_pool = random.sample(all_pools, 1)
        count_in_pool = pool_df[selected_pool][0]
        sum_current_group += count_in_pool
        current_group.append(selected_pool[0])
        all_pools.remove(selected_pool[0])
    pool_groups.append(current_group)
    #print(sum_current_group)
print('Total number of groups:', len(pool_groups), 'of ~', aprx_size_lists, 'samples')

Total number of groups: 17 of ~ 500 samples


## Export df as csv

In [11]:
#Sample
s.to_csv('Liao_sample_info.tsv',  sep='\t', index = False)

#Prep
#FMT
for group in range(0, len(fmt_pool_groups)):
    px = fmt[fmt['Pool'].isin(fmt_pool_groups[group])]
    name = 'prep/Liao_prep_fmt_' + str(group+1) + '.tsv'
    px.to_csv(name,  sep='\t', index = False)

#pt
pt.to_csv('prep/Liao_prep_pt.tsv',  sep='\t', index = False)

#Remaining Cases
for group in range(0, len(pool_groups)):
    px = p0[p0['Pool'].isin(pool_groups[group])]
    name = 'prep/Liao_prep_' + str(group+1) + '.tsv'
    px.to_csv(name,  sep='\t', index = False)